In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../data/cox-2_chembl.csv', sep=';')
df.head()

In [ ]:
df['Standard Units'].unique()

In [ ]:
df['Standard Units'] = df['Standard Units'].apply(lambda x: x if x == 'nM' else np.nan)

In [ ]:
df['Standard Units'].unique()

In [ ]:
df = df[['Molecule ChEMBL ID', 'Smiles', 'Standard Value', 'Standard Units']]
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
sorted_df = df.sort_values(by='Standard Value')

In [ ]:
sorted_df.head(10)

In [ ]:
sorted_unique = sorted_df.drop_duplicates(keep='first')
sorted_unique = sorted_unique.drop_duplicates(subset='Molecule ChEMBL ID', keep='first')
sorted_unique.shape

In [ ]:
import time
import random
from pathlib import Path

import pandas as pd
import numpy
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import Draw
from rdkit.Chem import rdFingerprintGenerator

In [ ]:
compounds = []
# .itertuples() returns a (index, column1, column2, ...) tuple per row
# we don't need index so we use _ instead
# note how we are slicing the dataframe to only the two columns we need now
for _, chembl_id, smiles in sorted_unique[["Molecule ChEMBL ID", "Smiles"]].itertuples():
    compounds.append((Chem.MolFromSmiles(smiles), chembl_id))

In [ ]:
compounds[2]

In [ ]:
# Create fingerprints for all molecules
rdkit_gen = rdFingerprintGenerator.GetRDKitFPGenerator(maxPath=5)
fingerprints = [rdkit_gen.GetFingerprint(mol) for mol, idx in compounds]

# How many compounds/fingerprints do we have?
print("Number of compounds converted:", len(fingerprints))
print("Fingerprint length per compound:", len(fingerprints[0]))

In [ ]:
def tanimoto_distance_matrix(fp_list):
    """Calculate distance matrix for fingerprint list"""
    dissimilarity_matrix = []
    # Notice how we are deliberately skipping the first and last items in the list
    # because we don't need to compare them against themselves
    for i in range(1, len(fp_list)):
        # Compare the current fingerprint against all the previous ones in the list
        similarities = DataStructs.BulkTanimotoSimilarity(fp_list[i], fp_list[:i])
        # Since we need a distance matrix, calculate 1-x for every element in similarity matrix
        dissimilarity_matrix.extend([1 - x for x in similarities])
    return dissimilarity_matrix

In [ ]:
sim = DataStructs.TanimotoSimilarity(fingerprints[3], fingerprints[3])
print(f"Tanimoto similarity: {sim:.2f}, distance: {1-sim:.2f}")

In [ ]:
tanimoto_distance_matrix(fingerprints)[0:8]

In [ ]:
from fixes import *

tanimoto_similarity_matrix(fingerprints[0:5]) - 1

In [ ]:
 def cluster_fingerprints(fingerprints, cutoff=0.2):
    """Cluster fingerprints
    Parameters:
        fingerprints
        cutoff: threshold for the clustering
    """
    # Calculate Tanimoto distance matrix
    distance_matrix = tanimoto_distance_matrix(fingerprints)
    # Now cluster the data with the implemented Butina algorithm:
    clusters = Butina.ClusterData(distance_matrix, len(fingerprints), cutoff, isDistData=True)
    clusters = sorted(clusters, key=len, reverse=True)
    return clusters

In [ ]:
# Run the clustering procedure for the dataset
clusters = cluster_fingerprints(fingerprints, cutoff=0.6)

# Give a short report about the numbers of clusters and their sizes
num_clust_g1 = sum(1 for c in clusters if len(c) == 1)
num_clust_g5 = sum(1 for c in clusters if len(c) > 5)
num_clust_g25 = sum(1 for c in clusters if len(c) > 25)
num_clust_g100 = sum(1 for c in clusters if len(c) > 100)

print("total # clusters: ", len(clusters))
print("# clusters with only 1 compound: ", num_clust_g1)
print("# clusters with >5 compounds: ", num_clust_g5)
print("# clusters with >25 compounds: ", num_clust_g25)
print("# clusters with >100 compounds: ", num_clust_g100)

In [ ]:
# Plot the size of the clusters
fig, ax = plt.subplots(figsize=(15, 4))
ax.set_xlabel("Cluster index")
ax.set_ylabel("Number of molecules")
ax.bar(range(1, len(clusters) + 1), [len(c) for c in clusters], lw=5);

In [ ]:
for cutoff in numpy.arange(0.0, 1.0, 0.2):
    clusters = cluster_fingerprints(fingerprints, cutoff=cutoff)
    fig, ax = plt.subplots(figsize=(15, 4))
    ax.set_title(f"Threshold: {cutoff:3.1f}")
    ax.set_xlabel("Cluster index")
    ax.set_ylabel("Number of molecules")
    ax.bar(range(1, len(clusters) + 1), [len(c) for c in clusters], lw=5)
    display(fig)

In [ ]:
cutoff = 0.2
clusters = cluster_fingerprints(fingerprints, cutoff=cutoff)

# Plot the size of the clusters - save plot
fig, ax = plt.subplots(figsize=(15, 4))
ax.set_xlabel("Cluster index")
ax.set_ylabel("# molecules")
ax.bar(range(1, len(clusters) + 1), [len(c) for c in clusters])
ax.set_title(f"Threshold: {cutoff:3.1f}")

print(
    f"Number of clusters: {len(clusters)} from {len(compounds)} molecules at distance cut-off {cutoff:.2f}"
)
print("Number of molecules in largest cluster:", len(clusters[0]))
print(
    f"Similarity between two random points in same cluster: {DataStructs.TanimotoSimilarity(fingerprints[clusters[0][0]], fingerprints[clusters[0][1]]):.2f}"
)
print(
    f"Similarity between two random points in different cluster: {DataStructs.TanimotoSimilarity(fingerprints[clusters[0][0]], fingerprints[clusters[1][0]]):.2f}"
)

In [ ]:
df.shape

In [ ]:
len(df['Molecule ChEMBL ID'].unique())

In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
from molecular_docking.smina_docking import docking_smina_single
prot_path = '/home/anton/PycharmProjects/cadd/data/prots/cox.pdbqt'
ref_lig = '/home/anton/PycharmProjects/cadd/data/prots/RCX_5kir.sdf'

In [ ]:
# docking_smina_single(protein_file=prot_path,
#                              ligand=sorted_unique.Smiles[7],
#                              autobox_ligand=prot_path,
#                              exhaustiveness=32,
#                              ligand_type='smiles',
#                              num_modes=5,
#                              docking_name='test_6_e32_whole',
#                              out_dir="test", )

In [ ]:
from fixes import Automaton

In [ ]:
test = Automaton()
test.load_chembl('../data/cox-2_chembl.csv')

In [ ]:
test.data.head()

In [ ]:
test.butina_cluster()

In [ ]:
for i in range(0,10, 1):
    cut = i / 10
    n_compounds = len(test.data)
    test.butina_cluster(cut)
    print(f'Number of cluster, if cutoff is {cut}: {len(test.clusters)}')
    cl_len = [len(i) for i in test.clusters]
    avg_cluster_size = sum(cl_len) / len(cl_len)
    num_clust_g1 = sum(1 for c in cl_len if c == 1)
    print(f'Average size of cluster is: {avg_cluster_size}')
    print(f'Number of single compound clusters: {num_clust_g1}')
    #print(f'Optimal cluster size: {n_compounds / len(test.clusters)}')
    print('----------')

In [ ]:
test.butina_cluster(0.5)

In [ ]:
test.clusters

In [ ]:
prob = 1 / len(test.clusters)

In [ ]:
prob

In [77]:
import numpy as np
cluster_idx = np.arange(0, len(test.clusters), 1)

In [78]:
len(test.clusters)

376

In [79]:
len(cluster_idx)

376

In [80]:
cluster_idx

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [81]:
initial_sample_cluster = np.random.choice(cluster_idx,replace=True, size=10)

In [82]:
initial_sample_cluster

array([160, 111, 163,  33, 163, 157,  91,  91, 122, 113])

In [ ]:
veri = []
for i in initial_sample_cluster:
    print(f'Cluster number {i}')
    inner_idx = np.arange(0,len(test.clusters[i]), 1)
    #print(inner_idx)
    sel_mol = np.random.choice(inner_idx, size=1)[0]
    original_id = test.clusters[i][sel_mol]
    print(f'Selected molecule: {original_id}')
    print(test.data.iloc[[original_id]].Smiles.values)

    test_smiles = test.data.iloc[[original_id]].Smiles.values[0]
    name = f'sample_{original_id}'
    docking_smina_single(protein_file=prot_path,
                             ligand=test_smiles,
                             autobox_ligand=prot_path,
                             exhaustiveness=32,
                             ligand_type='smiles',
                             num_modes=5,
                             docking_name=name,
                             out_dir="test", )
    veri.append(test.data.iloc[[original_id]].Smiles.values[0])
len(set(veri)) == len(veri)

Cluster number 160
Selected molecule: 1500
['Cc1ccc(NNC(=O)c2ccccc2)cc1']


1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders



./smina.static -r /home/anton/PycharmProjects/cadd/data/prots/cox.pdbqt -l ligand.sdf --autobox_ligand /home/anton/PycharmProjects/cadd/data/prots/cox.pdbqt --autobox_add 4 --num_modes 5  --exhaustiveness 32 --seed 42 -o test/sample_1500.sdf
   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 42

0%   10   20   30   40   50   60   70   80   90 

1 molecule converted
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders



./smina.static -r /home/anton/PycharmProjects/cadd/data/prots/cox.pdbqt -l ligand.sdf --autobox_ligand /home/anton/PycharmProjects/cadd/data/prots/cox.pdbqt --autobox_add 4 --num_modes 5  --exhaustiveness 32 --seed 42 -o test/sample_3446.sdf
   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 42

0%   10   20   30   40   50   60   70   80   90 

...

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 42

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
1       

1 molecule converted


./smina.static -r /home/anton/PycharmProjects/cadd/data/prots/cox.pdbqt -l ligand.sdf --autobox_ligand /home/anton/PycharmProjects/cadd/data/prots/cox.pdbqt --autobox_add 4 --num_modes 5  --exhaustiveness 32 --seed 42 -o test/sample_3606.sdf


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders



   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 42

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
*********************